### Mixture Density Network

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
from mdn import MixtureDensityNetwork, gmm_forward, get_argmax_mu, th2np

# --- 1. Parameters & Configuration ---

In [ ]:
# *** MODIFIED: This will now raise a FileNotFoundError if the file doesn't exist. ***
print("Loading training data 'training_data.npz'...")
train_data = np.load('../data1/noisy_data.npz')
x_train_np_orig = train_data['x']
y_train_np_orig = train_data['y']
print("Training data loaded successfully.")

N_SAMPLES = x_train_np_orig.shape[0]

print(f"Loaded {N_SAMPLES} training samples.")

Loading training data 'training_data.npz'...
Training data loaded successfully.
Loaded 800 training samples.


## --- 2. Data Preparation ---

In [3]:
# Reshape data for PyTorch
if x_train_np_orig.ndim == 1:
    x_train = x_train_np_orig.reshape(-1, 1)
else:
    x_train = x_train_np_orig

if y_train_np_orig.ndim == 1:
    y_train = y_train_np_orig.reshape(-1, 1)
else:
    y_train = y_train_np_orig
    
# Convert NumPy arrays to PyTorch Tensors
x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train).float()

# Create a TensorDataset and DataLoader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# --- 4. Model Init and Loss Function and Optimizer ---

In [4]:
print(f"Initializing MDN")
model = MixtureDensityNetwork(
    name='mdn_for_outliers',
    x_dim=1,
    y_dim=1,
    k=4, 
    h_dim_list=[64, 64], # Mimics our old MLP structure
    actv=nn.Tanh(),      # Use Tanh for a smoother function
    p_drop=0.0,           # Turn off dropout for this simple problem
    sig_max    = 3.0,
    mu_min     = -2.5,
    mu_max     = +2.5
)
print(model)

# --- 4. Optimizer ---

# We DON'T define a 'criterion' (loss) here, it's calculated in the loop.
# We can still use weight_decay (L2 regularization) on the base network
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

Initializing MDN
MixtureDensityNetwork(
  (actv): Tanh()
  (net): Sequential(
    (linear_00): Linear(in_features=1, out_features=64, bias=True)
    (tanh_01): Tanh()
    (linear_02): Linear(in_features=64, out_features=64, bias=True)
    (tanh_03): Tanh()
    (mixturesofgaussianlayer_04): MixturesOfGaussianLayer(
      (fc_pi): Linear(in_features=64, out_features=4, bias=True)
      (fc_mu): Linear(in_features=64, out_features=4, bias=True)
      (fc_sigma): Linear(in_features=64, out_features=4, bias=True)
    )
  )
)


# --- 5. Training Loop ---

In [ ]:
print("Starting MDN training...")
# Training Parameters
NUM_EPOCHS = 1000
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    for x_batch, y_batch in train_loader:
        
        # 1. Forward pass: Get the GMM parameters (pi, mu, sigma)
        pi_batch, mu_batch, sigma_batch = model(x_batch)
        
        # 2. Compute Loss: Calculate the Negative Log-Likelihood
        #    We use the 'gmm_forward' function we lifted
        out_dict = gmm_forward(pi_batch, mu_batch, sigma_batch, y_batch)
        nlls = out_dict['nlls'] # nlls is a tensor of shape [N]
        loss = torch.mean(nlls) # Our loss is the average NLL
        
        # 3. Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Print progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], NLL Loss: {epoch_loss/len(train_loader):.4f}')

print("Training finished.")

Starting MDN training...
Epoch [100/1000], NLL Loss: -0.0717
Epoch [200/1000], NLL Loss: -0.1411
Epoch [300/1000], NLL Loss: -0.2315
Epoch [400/1000], NLL Loss: -0.2366
Epoch [500/1000], NLL Loss: -0.2934
Epoch [600/1000], NLL Loss: -0.3380
Epoch [700/1000], NLL Loss: -0.3595
Epoch [800/1000], NLL Loss: -0.3802
Epoch [900/1000], NLL Loss: -0.2892
Epoch [1000/1000], NLL Loss: -0.3541
Training finished.


# --- 6. Visualization with Validation Data ---

In [6]:
print("Loading validation data 'validation_data.npz'...")
val_data = np.load('../data1/clean_data.npz')
x_val = val_data['x']
y_val = val_data['y']
print("Validation data loaded successfully.")
    
# Prepare validation data (reshape and convert to tensor)
if x_val.ndim == 1: x_val = x_val.reshape(-1, 1)
if y_val.ndim == 1: y_val = y_val.reshape(-1, 1)

x_val_tensor = torch.tensor(x_val).float()

# Set model to evaluation mode
model.eval()

# Get model predictions for the validation set
with torch.no_grad():
    # 1. Get the full GMM parameters
    pi_val, mu_val, sigma_val = model(x_val_tensor)
    
    # 2. Get the *most probable* mean (our final prediction)
    #    This is the key! We are no longer using a single, overfitted output.
    y_val_pred_tensor = get_argmax_mu(pi_val, mu_val)

# Convert all data back to NumPy for plotting
y_val_pred_np = th2np(y_val_pred_tensor)
x_val_np = th2np(x_val_tensor)
y_val_np = y_val # Already a numpy array

# Create the scatter plot
plt.figure(figsize=(9, 6))

# Plot the original training data (with noise)
plt.scatter(x_train_np_orig, y_train_np_orig, color='green', label='Noisy Training Data', s=10, alpha=0.2)

# Plot the actual validation data (the "ground truth")
plt.scatter(x_val_np, y_val_np, color='blue', label='Actual Validation Data', s=15, alpha=0.9)

# Plot the model's predictions (The most probable mean)
plt.scatter(x_val_np, y_val_pred_np, color='red', label='Model Predictions (Argmax Mu)', s=15, alpha=0.9)

# Plot the prediction line
sort_indices = np.argsort(x_val_np.squeeze())
plt.plot(x_val_np[sort_indices], y_val_pred_np[sort_indices], color='red', linestyle='--', linewidth=2.5, label='Model Prediction Line')

plt.title('MDN Prediction (Argmax Mu) vs. All Data')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.ylim(y_train_np_orig.min() - 0.5, y_train_np_orig.max() + 0.5) # Set Y-limits
plt.xlim(x_train_np_orig.min() - 0.1, x_train_np_orig.max() + 0.1) # Set X-limits
plt.show()

Loading validation data 'validation_data.npz'...
Validation data loaded successfully.


NameError: name 'th2np' is not defined